# Import

In [1]:
import pandas as pd
from sodapy import Socrata
from api_keys import *
import datetime

In [2]:
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("www.dallasopendata.com", app_token)

In [3]:
#grab data from api
#sdr7-6v3j is the dataset code from sodapy
all_data = client.get_all("sdr7-6v3j")

#turn into pandas df
data_df = pd.DataFrame.from_records(all_data)

In [4]:
data_df.sort_values(by='ararrestdate',ascending=False, inplace=True)

In [5]:
#looking for only certain columns
data_df = data_df[['incidentnum', 'arrestnumber', 
                   'ararrestdate', 'ararresttime', 
                   'arpremises', 'arladdress', 
                   'arlzip','sex','drugrelated',
                   'drugtype','age']]


In [6]:
# reduced to rows with drugrelated = yes or uknown
data_df = data_df.loc[((data_df['drugrelated']=='Yes') | (data_df['drugrelated']=='Uknown'))]
data_df.head(3)

,incidentnum,arrestnumber,ararrestdate,ararresttime,arpremises,arladdress,arlzip,sex,drugrelated,drugtype,age
937,202924-2022,22-032078,2022-11-09T00:00:00.000,00:30,Outdoor Area Public/Private,1400 E JEFFERSON BLVD,75203,Male,Yes,Methamphetamine,33
1166,203531-2022,22-032163,2022-11-09T00:00:00.000,23:50,"Highway, Street, Alley ETC",100 S MASTERS DR,75217,Female,Yes,Oxycodone,49
944,203511-2022,22-032161,2022-11-09T00:00:00.000,22:48,"Highway, Street, Alley ETC",2100 W NORTHWEST HWY,75220,Male,Yes,Crack Cocaine,48


In [7]:
#this edits the ararrestdate so it doesn't include the time 
for index, row in data_df.iterrows():
        split = row['ararrestdate'].split('T')
        row['ararrestdate'] = split[0]
data_df.head()

,incidentnum,arrestnumber,ararrestdate,ararresttime,arpremises,arladdress,arlzip,sex,drugrelated,drugtype,age
937,202924-2022,22-032078,2022-11-09,00:30,Outdoor Area Public/Private,1400 E JEFFERSON BLVD,75203,Male,Yes,Methamphetamine,33
1166,203531-2022,22-032163,2022-11-09,23:50,"Highway, Street, Alley ETC",100 S MASTERS DR,75217,Female,Yes,Oxycodone,49
944,203511-2022,22-032161,2022-11-09,22:48,"Highway, Street, Alley ETC",2100 W NORTHWEST HWY,75220,Male,Yes,Crack Cocaine,48
1130,203409-2022,22-032136,2022-11-09,18:44,NaN,13000 COIT RD,75240,Male,Uknown,NaN,31
1112,202764-2022,22-032046,2022-11-08,17:57,Parking Lot (Park),900 SAN JACINTO ST,75202,Male,Uknown,NaN,20


In [8]:
#the connecting to sqlite begins
import sqlite3

In [9]:
#connection string
conn =sqlite3.connect('test_database')
c = conn.cursor()

In [10]:
#creating the table 
c.execute('CREATE TABLE IF NOT EXISTS crime_test (incidentnum, arrestnumber, ararrestdate, ararresttime, arpremises, arladdress, arlzip, sex, drugrelated, drugtype, age)')
conn.commit()

In [11]:
#exports the dataframe to sqlite
data_df.to_sql('crime_test', conn, if_exists='replace', index = False)

20623

In [12]:
c.execute('''select * from crime_test''')

In [13]:
#viewing most recent row of the table
for row in c.fetchall():
    print(row)
    break

('202924-2022', '22-032078', '2022-11-09', '00:30', 'Outdoor Area Public/Private', '1400 E JEFFERSON BLVD', '75203', 'Male', 'Yes', 'Methamphetamine', '33')


In [ ]:
#decrease the amount of data to filter by today or past week, month, and year
#creating the variables to filter through the df's
today = datetime.date.today()
week = today - datetime.timedelta(days=7)
month = today - datetime.timedelta(days=30)
year = (today - datetime.timedelta(days=365)).strftime('%Y-%m-%d')
today = today.strftime('%Y-%m-%d')
week = week.strftime('%Y-%m-%d')
month = month.strftime('%Y-%m-%d')

In [ ]:
#filter through the dataframes descending to not constantly work with big data for each filter
year_df = data_df[(data_df['ararrestdate'] >= year)]
month_df = year_df[(year_df['ararrestdate'] >= month)]
week_df = month_df[(month_df['ararrestdate'] >= week)]
today_df = week_df[(week_df['ararrestdate'] >= today)]

In [ ]:
week_df

In [ ]:
from geopy.geocoders import Nominatim


In [ ]:
geolocator = Nominatim(user_agent="test_crime_data_app")
location = geolocator.geocode("8550 N STEMMONS SERV NB")

print((location.latitude, location.longitude))

In [ ]:
# loop through each rows address and zip code, IF address + zip code gives lat/long append, 
#ELIF address gives lat/long append, elif get zip code lat/lng, else no zip/address put NaN for lat/lng
lat_lngs = {'lat':[],'lng':[]}

In [ ]:
#retrieving lat and longs for leaflet maps
for index, row in year_df.iterrows():
    test = ''
    test += row['arladdress']
    test += ' ' + row['arlzip']
    try:
        location = geolocator.geocode(test)
        lat_lngs['lat'].append(location.latitude)
        lat_lngs['lng'].append(location.longitude)
    except:
        try:
            test = test[:-5]
            location = geolocator.geocode(test)
            lat_lngs['lat'].append(location.latitude)
            lat_lngs['lng'].append(location.longitude)
        except:
            try:
                location = geolocator.geocode(row['arlzip'])
                lat_lngs['lat'].append(location.latitude)
                lat_lngs['lng'].append(location.longitude)
            except:
                location = ['NaN', 'Nan']
                lat_lngs['lat'].append(location.latitude)
                lat_lngs['lng'].append(location.longitude)

In [ ]:
lat_lng_df = pd.DataFrame(lat_lngs)

In [ ]:
year_df.reset_index(inplace=True)

In [ ]:
week_df.reset_index(inplace=True)

In [ ]:
week_df.head()

In [ ]:
merged = pd.concat([year_df, lat_lng_df],axis=1)

In [ ]:
test_df = merged.loc[merged['lat'] == 'NaN']

In [ ]:
merged.head()